In [217]:
import pandas as pd

tracks_df = pd.read_json("./../data/raw/05_02_v2/tracks.jsonl", lines=True)
users_df = pd.read_json("./../data/raw/05_02_v2/users.jsonl", lines=True)
session_df = pd.read_json("./../data/raw/05_02_v2/sessions.jsonl", lines=True)
artists_df = pd.read_json("./../data/raw/05_02_v2/artists.jsonl", lines=True)

tracks_df.columns, users_df.columns, session_df.columns

(Index(['id', 'name', 'popularity', 'duration_ms', 'explicit', 'id_artist',
        'release_date', 'danceability', 'energy', 'key', 'loudness',
        'speechiness', 'acousticness', 'instrumentalness', 'liveness',
        'valence', 'tempo'],
       dtype='object'),
 Index(['user_id', 'name', 'city', 'street', 'favourite_genres',
        'premium_user'],
       dtype='object'),
 Index(['session_id', 'timestamp', 'user_id', 'track_id', 'event_type'], dtype='object'))

In [168]:
tracks_df[tracks_df["id"] == "40f52Y5P5PDfAxu2P3iI3c"]

,id,name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
19295,40f52Y5P5PDfAxu2P3iI3c,A Gente Não Se Aguenta - Ao Vivo,53,148280,0,1yR65psqiazQpeM79CcGh8,2017-03-06,0.67,0.862,11,-5.438,0.0496,0.224,0.000002,0.32,0.931,172.076


# Stats for pairs track-user

In [215]:
# get the track-user stats


def agg_events(x):
    return tuple(x)


tracks_tmp = tracks_df[["id", "duration_ms"]]
# 13918
# session_tmp = session_df[session_df['session_id'].isin((125, 126))].sort_values(by=['session_id', 'timestamp'], ascending=[True, True])
session_tmp = session_df.sort_values(
    by=["session_id", "timestamp"], ascending=[True, True]
)

session_tmp["time_played"] = (
    pd.to_datetime(session_tmp["timestamp"]).diff().dt.total_seconds() * 1000
)
session_tmp["time_played"] = session_tmp["time_played"].shift(-1)

track_minutes_played = (
    session_tmp.groupby(["track_id", "user_id"])["time_played"].agg("sum").reset_index()
)
grouped_events = (
    session_tmp.groupby(["track_id", "user_id"])["event_type"]
    .agg([agg_events])
    .reset_index()
)

track_user_stats = pd.merge(
    track_minutes_played, tracks_tmp, how="left", left_on="track_id", right_on="id"
)
track_user_stats = pd.merge(
    track_user_stats, grouped_events, how="left", on=["track_id", "user_id"]
)

track_user_stats["percentage_played"] = (
    track_user_stats["time_played"] / track_user_stats["duration_ms"] * 100
)
track_user_stats["percentage_played"] = track_user_stats.apply(
    lambda row: 100.0 if row.percentage_played > 100 else row.percentage_played, axis=1
)
track_user_stats["was_liked"] = track_user_stats.apply(
    lambda row: "like" in row.agg_events, axis=1
)
track_user_stats["was_skipped"] = track_user_stats.apply(
    lambda row: "skip" in row.agg_events, axis=1
)

track_user_stats = track_user_stats.drop(columns=["id", "agg_events"])
track_user_stats = track_user_stats.drop(
    track_user_stats[
        abs(track_user_stats.time_played) / 10 > track_user_stats.duration_ms
    ].index
)
track_user_stats = track_user_stats.drop(
    track_user_stats[track_user_stats.time_played == 0.0].index
)
track_user_stats

,track_id,user_id,time_played,duration_ms,percentage_played,was_liked,was_skipped
1,000xYdQfIZ4pDmBGzQalKU,155,187119.0,187119,100.000000,False,False
2,000xYdQfIZ4pDmBGzQalKU,187,134152.0,187119,71.693414,False,True
3,000xYdQfIZ4pDmBGzQalKU,194,192119.0,187119,100.000000,False,False
4,000xYdQfIZ4pDmBGzQalKU,203,188119.0,187119,100.000000,True,False
5,000xYdQfIZ4pDmBGzQalKU,220,192119.0,187119,100.000000,False,False
...,...,...,...,...,...,...,...
412784,7zzoxJbgjme3366mOp5UnH,497,231284.0,280800,82.366097,False,True
412785,7zzoxJbgjme3366mOp5UnH,509,285800.0,280800,100.000000,False,False
412786,7zzoxJbgjme3366mOp5UnH,524,168890.0,280800,60.146011,True,True
412787,7zzoxJbgjme3366mOp5UnH,537,263042.0,280800,93.675926,False,True


# Listing how many times a track was liked

In [235]:
liked_tracks = track_user_stats
liked_tracks["times_liked"] = 0

liked_tracks = liked_tracks[liked_tracks["was_liked"] == True]
liked_tracks = (
    liked_tracks.groupby(["track_id"])["times_liked"].agg("count").reset_index()
)
liked_tracks

,track_id,times_liked
0,000xYdQfIZ4pDmBGzQalKU,4
1,003FTlCpBTM4eSqYSWPv4H,6
2,003vvx7Niy0yvhvHt4a68B,5
3,005Dlt8Xaz3DkaXiRJgdiS,3
4,005lwxGU1tms6HGELIcUv9,3
...,...,...
21891,7zv5oGRQoJX09QM3pkvI1w,3
21892,7zwn1eykZtZ5LODrf7c0tS,4
21893,7zycSpvjDcqh6YT1FEl2kY,2
21894,7zzhknA0A39TH81meuX7WA,4


# Listing how many times a track was skipped

In [238]:
skipped_tracks = track_user_stats
skipped_tracks["times_skipped"] = 0

skipped_tracks = skipped_tracks[skipped_tracks["was_skipped"] == True]
skipped_tracks = (
    skipped_tracks.groupby(["track_id"])["times_skipped"].agg("count").reset_index()
)
skipped_tracks

,track_id,times_skipped
0,000xYdQfIZ4pDmBGzQalKU,4
1,003FTlCpBTM4eSqYSWPv4H,4
2,003vvx7Niy0yvhvHt4a68B,10
3,005Dlt8Xaz3DkaXiRJgdiS,3
4,005lwxGU1tms6HGELIcUv9,7
...,...,...
22328,7zv5oGRQoJX09QM3pkvI1w,4
22329,7zwn1eykZtZ5LODrf7c0tS,5
22330,7zycSpvjDcqh6YT1FEl2kY,6
22331,7zzhknA0A39TH81meuX7WA,1


# Genres listed by how many times songs from these genres were played

In [228]:
tracks_genre_df = tracks_df.merge(
    artists_df[["id", "genres"]], left_on="id_artist", right_on="id", how="inner"
)
exploded_genres = tracks_genre_df[["id_x", "genres"]].explode("genres").reset_index()
exploded_genres = exploded_genres.rename(
    columns={"id_x": "track_id", "genres": "genre"}
)
session_df_with_genres = session_df.merge(
    exploded_genres, left_on="track_id", right_on="track_id", how="inner"
)

session_df_with_genres["genres_count"] = 0
grouped_genres = (
    session_df_with_genres.groupby(["genre"])["genres_count"].agg("count").reset_index()
)
grouped_genres = grouped_genres.sort_values(by="genres_count", ascending=False)
grouped_genres

,genre,genres_count
525,pop,137948
613,rock,114560
167,dance pop,81029
551,post-teen pop,68595
383,latin,60227
...,...,...
164,crunk,21
286,glitchcore,21
231,experimental hip hop,20
560,psychedelic hip hop,20
